In [1]:
import os, sys
from datetime import datetime
import time
from util import adjacency_matrix_to_nei_list, dirichlet_sampling_balanced, \
    get_adjacency_matrix, generate_node_configs, generate_attack_matrix, load_dataset

from evaluation_util import read_experiment_csvs
from local_node import local_node
import pickle

from fed_avg import fed_avg
from krum import krum
from trimmedmean import trimmedMean
from median import median
import copy 

import logging

In [2]:
# configure logging at the root level of Lightning
logging.getLogger("lightning.pytorch").setLevel(logging.ERROR)
# configure logging on module level, redirect to file
logger = logging.getLogger("lightning.pytorch.core")
logger.addHandler(logging.FileHandler("core.log"))
log = logging.getLogger("pytorch_lightning")
log.propagate = False
log.setLevel(logging.ERROR)

In [3]:
from lightning.pytorch.loggers import CSVLogger
from torch.utils.data import Subset
import random
import numpy as np
from poisoning_attack import modelpoison
import copy
from util import cosine_metric, cosine_metric2, manhattan_metric, chebyshev_metric, pearson_correlation_metric, euclidean_metric
from sklearn.cluster import DBSCAN

In [4]:
from mnistmodel import MNISTModelMLP
from fmnistmodel import FashionMNISTModelMLP
from cifar10model import SimpleMobileNet

In [5]:
from data_util import DynamicDataLoader, DynamicDataset, dynamic_transformer
from itertools import product

In [6]:
cwd = os.getcwd()

In [7]:
import random
from subset import ChangeableSubset
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Subset

In [8]:
def get_datloader(train_dataset,test_dataset, num_peers, indices, data_poisoning, poisoned_sample_ratio, targeted):
    tr_subset = ChangeableSubset(train_dataset, indices, data_poisoning=data_poisoning, 
                                    poisoned_sample_ratio=poisoned_sample_ratio, targeted=targeted,
                                    target_label=3, target_changed_label=7, noise_type="salt")

    data_train, data_val = random_split(
                tr_subset,
                [
                    int(len(tr_subset) * 0.8),
                    len(tr_subset) - int(len(tr_subset) * 0.8),
                ],
            )
    data_train_loader = DataLoader(data_train, batch_size=64,shuffle=True)
    data_val_loader = DataLoader(data_val, batch_size=64,shuffle=False)

    number_test = int(len(test_dataset)/num_peers)
    test_indices = random.sample(range(len(test_dataset)), number_test)   


    number_backdoor_valid = int(number_test*0.2)
    test_backdoor_valid_indices = random.sample(test_indices, number_backdoor_valid)
    test_indices = list(set(test_indices) - set(test_backdoor_valid_indices))
    test_backdoor_valid = ChangeableSubset(test_dataset, test_backdoor_valid_indices, label_flipping=False, 
                                            data_poisoning=True, poisoned_sample_ratio=100, noise_injected_ratio=100, 
                                            targeted=True, target_label=3, target_changed_label=7, noise_type="salt", 
                                            backdoor_validation=True)

    backdoor_valid_loader = DataLoader(test_backdoor_valid, batch_size=64,shuffle=False)

    test_dataset = Subset(test_dataset, test_indices) 
     
    return data_train_loader, data_val_loader, backdoor_valid_loader, test_dataset

               

In [9]:
from lightning.pytorch.loggers import CSVLogger

first experiment, benign_node vs benign_node, benign_node vs backdoored_node

In [10]:
dataset_name = "MNIST"
maxEpoch = 30
alpha = 100
attack_type = 'sample poisoning'
maxEpoch = 30
poisoned_sample_ratio = 100
targeted = True
num_peers = 10
train_dataset, test_dataset = load_dataset(dataset_name)
targets = train_dataset.targets
client_indices = dirichlet_sampling_balanced(targets, alpha, num_peers)
benige_indices_0 = client_indices[0]
benige_indices_1 = client_indices[1]
backdoor_indices_0 = client_indices[0]

In [11]:
data_train_loader_b0, data_val_loader_b0, backdoor_valid_loader_b0, test_dataset_b0 = get_datloader(train_dataset, test_dataset, num_peers,benige_indices_0, data_poisoning=False, poisoned_sample_ratio=0, targeted=False)
data_train_loader_b1, data_val_loader_b1, backdoor_valid_loader_b1, test_dataset_b1 = get_datloader(train_dataset, test_dataset, num_peers,benige_indices_1, data_poisoning=False, poisoned_sample_ratio=0, targeted=False)
data_train_loader_m1, data_val_loader_m1, backdoor_valid_loader_m1, test_dataset_m1 = get_datloader(train_dataset, test_dataset, num_peers,backdoor_indices_0, data_poisoning=True, poisoned_sample_ratio=100, targeted=True)

Datapoisoning: generating watermarked samples for testing (all classes)
Datapoisoning: generating watermarked samples for testing (all classes)
Datapoisoning: generating watermarked samples for training, target: 3


j:\git\beidou\poisoning_attack.py:151: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(img).clone().detach()


Datapoisoning: generating watermarked samples for testing (all classes)


In [12]:
dataset_name = "MNIST"
maxRound = 1
maxEpoch = 20
alpha = 100
attack_type = ['no attack','no attack','sample poisoning','sample poisoning']
poisoned_sample_ratio = 100
targeted = True
num_peers = 10

In [13]:
def train_backdoor(dataset, alpha, maxEpoch):
    cwd = os.getcwd()
    experimentsName=f'{dataset_name}_BACKDOOR_alpha_{alpha}'
    experimentsName_path = cwd+'/backdoor_experiments/'+experimentsName
    train_dataset, test_dataset = load_dataset(dataset_name)
    targets = train_dataset.targets
    client_indices = dirichlet_sampling_balanced(targets, alpha, num_peers)
    node_list = {}
    for i in range(4):
        node_id = i
        node_config = generate_node_configs(i, client_indices[i], experimentsName, experimentsName_path, 
                                            dataset_name, [], num_peers, maxRound, maxEpoch, 
                                            train_dataset, test_dataset, attack_type[i], targeted, 
                                            100, poisoned_sample_ratio, fed_avg,
                                            False, False, False, False)
        
        basic_config = node_config['basic_config']
        data_train_loader = node_config['data_train_loader']
        data_val_loader = node_config['data_val_loader']
        test_dataset_loader = node_config['test_dataset_loader']
        backdoor_valid_loader = node_config['backdoor_valid_loader']

        node = local_node(node_id,basic_config, data_train_loader, data_val_loader, test_dataset_loader, backdoor_valid_loader)
        node_list[node_id] = node
        with open(experimentsName_path+f"/{node_id}_config.pk", "wb") as f:
            pickle.dump(node_config, f)
            f.close()
        node.local_training()

In [15]:
train_backdoor(dataset_name, alpha, maxEpoch) 

Datapoisoning: generating watermarked samples for testing (all classes)
Epoch end: Validation, epoch number: 0
Epoch end: Validation, epoch number: 1
Epoch end: Train, epoch number: 0
Epoch end: Validation, epoch number: 2
Epoch end: Train, epoch number: 1
Epoch end: Validation, epoch number: 3
Epoch end: Train, epoch number: 2
Epoch end: Validation, epoch number: 4
Epoch end: Train, epoch number: 3
Epoch end: Validation, epoch number: 5
Epoch end: Train, epoch number: 4
Epoch end: Validation, epoch number: 6
Epoch end: Train, epoch number: 5
Epoch end: Validation, epoch number: 7
Epoch end: Train, epoch number: 6
Epoch end: Validation, epoch number: 8
Epoch end: Train, epoch number: 7
Epoch end: Validation, epoch number: 9
Epoch end: Train, epoch number: 8
Epoch end: Validation, epoch number: 10
Epoch end: Train, epoch number: 9
Epoch end: Validation, epoch number: 11
Epoch end: Train, epoch number: 10
Epoch end: Validation, epoch number: 12
Epoch end: Train, epoch number: 11
Epoch en

j:\git\beidou\venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch end: Test, epoch number: 0


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       Test/Accuracy       │    0.8528026342391968     │
│       Test/F1Score        │    0.8054435849189758     │
│         Test/Loss         │    2.9777603149414062     │
│      Test/Precision       │    0.7788698673248291     │
│        Test/Recall        │    0.8528026342391968     │
│    TestEpoch/Accuracy     │    0.8526278734207153     │
│     TestEpoch/F1Score     │    0.8090451955795288     │
│    TestEpoch/Precision    │     0.778031051158905     │
│     TestEpoch/Recall      │    0.8526278734207153     │
└───────────────────────────┴───────────────────────────┘

Datapoisoning: generating watermarked samples for testing (all classes)


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       Test/Accuracy       │    0.8498977422714233     │
│       Test/F1Score        │    0.8063822388648987     │
│         Test/Loss         │    1.0457643270492554     │
│      Test/Precision       │     0.785552442073822     │
│        Test/Recall        │    0.8498977422714233     │
│    TestEpoch/Accuracy     │    0.8576279878616333     │
│     TestEpoch/F1Score     │    0.8161615133285522     │
│    TestEpoch/Precision    │    0.7863971590995789     │
│     TestEpoch/Recall      │    0.8576279878616333     │
└───────────────────────────┴───────────────────────────┘

Datapoisoning: generating watermarked samples for training, target: 3
Datapoisoning: generating watermarked samples for testing (all classes)


j:\git\beidou\venv\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:653: Checkpoint directory J:\git\beidou\backdoor_experiments\MNIST_BACKDOOR_alpha_100\node_3 exists and is not empty.


Epoch end: Validation, epoch number: 0
Epoch end: Validation, epoch number: 1
Epoch end: Train, epoch number: 0
Epoch end: Validation, epoch number: 2
Epoch end: Train, epoch number: 1
Epoch end: Validation, epoch number: 3
Epoch end: Train, epoch number: 2
Epoch end: Validation, epoch number: 4
Epoch end: Train, epoch number: 3
Epoch end: Validation, epoch number: 5
Epoch end: Train, epoch number: 4
Epoch end: Validation, epoch number: 6
Epoch end: Train, epoch number: 5
Epoch end: Validation, epoch number: 7
Epoch end: Train, epoch number: 6
Epoch end: Validation, epoch number: 8
Epoch end: Train, epoch number: 7
Epoch end: Validation, epoch number: 9
Epoch end: Train, epoch number: 8
Epoch end: Validation, epoch number: 10
Epoch end: Train, epoch number: 9
Epoch end: Validation, epoch number: 11
Epoch end: Train, epoch number: 10
Epoch end: Validation, epoch number: 12
Epoch end: Train, epoch number: 11
Epoch end: Validation, epoch number: 13
Epoch end: Train, epoch number: 12
Epoch

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       Test/Accuracy       │    0.8455999493598938     │
│       Test/F1Score        │    0.7911894917488098     │
│         Test/Loss         │     1.180232048034668     │
│      Test/Precision       │    0.7666541337966919     │
│        Test/Recall        │    0.8455999493598938     │
│    TestEpoch/Accuracy     │    0.8538813591003418     │
│     TestEpoch/F1Score     │    0.8017850518226624     │
│    TestEpoch/Precision    │    0.7658383846282959     │
│     TestEpoch/Recall      │    0.8538813591003418     │
└───────────────────────────┴───────────────────────────┘

In [16]:
for dataset_name in ["MNIST", "FashionMNIST", "Cifar10"]:
    for alpha in [100, 10, 1, 0.5, 0.1]:
        if dataset_name == "Cifar10":
            maxEpoch = 30
        else:
            maxEpoch = 20            
        train_backdoor(dataset_name, alpha, maxEpoch)
        time.sleep(5)

Datapoisoning: generating watermarked samples for testing (all classes)


j:\git\beidou\poisoning_attack.py:151: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(img).clone().detach()
j:\git\beidou\venv\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:653: Checkpoint directory J:\git\beidou\backdoor_experiments\MNIST_BACKDOOR_alpha_100\node_0 exists and is not empty.
j:\git\beidou\venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
j:\git\beidou\venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_worke

Epoch end: Validation, epoch number: 0
Epoch end: Validation, epoch number: 1
Epoch end: Train, epoch number: 0
Epoch end: Validation, epoch number: 2
Epoch end: Train, epoch number: 1
Epoch end: Validation, epoch number: 3
Epoch end: Train, epoch number: 2
Epoch end: Validation, epoch number: 4
Epoch end: Train, epoch number: 3
Epoch end: Validation, epoch number: 5
Epoch end: Train, epoch number: 4
Epoch end: Validation, epoch number: 6
Epoch end: Train, epoch number: 5
Epoch end: Validation, epoch number: 7
Epoch end: Train, epoch number: 6
Epoch end: Validation, epoch number: 8
Epoch end: Train, epoch number: 7
Epoch end: Validation, epoch number: 9
Epoch end: Train, epoch number: 8
Epoch end: Validation, epoch number: 10
Epoch end: Train, epoch number: 9
Epoch end: Validation, epoch number: 11
Epoch end: Train, epoch number: 10
Epoch end: Validation, epoch number: 12
Epoch end: Train, epoch number: 11
Epoch end: Validation, epoch number: 13
Epoch end: Train, epoch number: 12
Epoch

j:\git\beidou\venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch end: Test, epoch number: 0


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       Test/Accuracy       │    0.1967858523130417     │
│       Test/F1Score        │    0.07793699949979782    │
│         Test/Loss         │    7.1832427978515625     │
│      Test/Precision       │    0.07235561311244965    │
│        Test/Recall        │    0.1967858523130417     │
│    TestEpoch/Accuracy     │    0.1984812468290329     │
│     TestEpoch/F1Score     │    0.08296477794647217    │
│    TestEpoch/Precision    │    0.08931136876344681    │
│     TestEpoch/Recall      │    0.1984812468290329     │
└───────────────────────────┴───────────────────────────┘

Datapoisoning: generating watermarked samples for training, target: 3
Datapoisoning: generating watermarked samples for testing (all classes)


j:\git\beidou\venv\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:653: Checkpoint directory J:\git\beidou\backdoor_experiments\Cifar10_BACKDOOR_alpha_0.1\node_3 exists and is not empty.


Epoch end: Validation, epoch number: 0
Epoch end: Validation, epoch number: 1
Epoch end: Train, epoch number: 0
Epoch end: Validation, epoch number: 2
Epoch end: Train, epoch number: 1
Epoch end: Validation, epoch number: 3
Epoch end: Train, epoch number: 2
Epoch end: Validation, epoch number: 4
Epoch end: Train, epoch number: 3
Epoch end: Validation, epoch number: 5
Epoch end: Train, epoch number: 4
Epoch end: Validation, epoch number: 6
Epoch end: Train, epoch number: 5
Epoch end: Validation, epoch number: 7
Epoch end: Train, epoch number: 6
Epoch end: Validation, epoch number: 8
Epoch end: Train, epoch number: 7
Epoch end: Validation, epoch number: 9
Epoch end: Train, epoch number: 8
Epoch end: Validation, epoch number: 10
Epoch end: Train, epoch number: 9
Epoch end: Validation, epoch number: 11
Epoch end: Train, epoch number: 10
Epoch end: Validation, epoch number: 12
Epoch end: Train, epoch number: 11
Epoch end: Validation, epoch number: 13
Epoch end: Train, epoch number: 12
Epoch

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       Test/Accuracy       │    0.12069899588823318    │
│       Test/F1Score        │    0.0491594634950161     │
│         Test/Loss         │     7.641689300537109     │
│      Test/Precision       │    0.0680886059999466     │
│        Test/Recall        │    0.12069899588823318    │
│    TestEpoch/Accuracy     │    0.12073171138763428    │
│     TestEpoch/F1Score     │    0.05112302303314209    │
│    TestEpoch/Precision    │    0.0787096843123436     │
│     TestEpoch/Recall      │    0.12073171138763428    │
└───────────────────────────┴───────────────────────────┘

In [ ]:
node.logger.log_metrics({"Test/ASR-backdoor": 0.9828571428571429})

In [ ]:
node.node_id

3